In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils3
import tile_extraction_v3

In [2]:
df = pd.read_csv('train_subset.csv')

In [3]:
# for i in range(12):
#     f, axarr = plt.subplots(1, 3, figsize=(20,20))
#     idx = np.random.randint(0, len(df))        
#     row = df.loc[idx]
#     im = utils3.imread(f"{tile_extraction_v3.IMG_DIR}{row['image_id']}.tiff", layer=tile_extraction_v3.LAYER)        
#     arr = np.asarray(im)
#     arr = utils3.enhance_image(arr)
#     arr = utils3.unsharp_masking(arr)
#     tiles1, img_map = tile_extraction_v3.akensert_tiles(arr, debug=True)
#     tiles2 = tile_extraction_v3.lafoss_tiles(arr)    
#     axarr[0].set_title(f'id:{idx}')
#     axarr[0].imshow(img_map.squeeze())
#     axarr[1].set_title('akensert')
#     axarr[1].imshow(tiles1.squeeze())
#     axarr[2].set_title('lafoss')
#     axarr[2].imshow(tiles2.squeeze()) 

In [4]:
import os
from multiprocessing import Pool
from tqdm.notebook import tqdm
from PIL import Image
import time
import utils3
tile_extractor= tile_extraction_v3.TileExtractor(1,64,4,4,0)
if(not os.path.exists(tile_extractor.tiles_dir)):
    os.mkdir(tile_extractor.tiles_dir)

new_df= pd.DataFrame(columns=df.columns)
for index,row in df.iterrows():
    new_df= new_df.append(row.copy(deep=True),ignore_index=True)
    for i in range(tile_extractor.augmentation):
        new_row=row.copy(deep=True)
        new_row['image_id']="{}_{}".format(new_row['image_id'],i)
        new_df= new_df.append(new_row,ignore_index=True)
        
new_df.to_csv("{}.csv".format(tile_extractor.tiles_dir),index=False)
#tile_extraction_v3.process_image(df['image_id'][0])

batch = df['image_id']
with Pool(processes=8) as pool:
    res = list(
        tqdm(pool.imap(tile_extractor.process_image, list(batch)), total = len(batch))
    )

  0%|          | 0/605 [00:00<?, ?it/s]